In [1]:
"""Run inference on a given lmdb dataset."""

import argparse
import sys
import numpy as np
from pathlib import Path

from ocpmodels.datasets.lmdb_dataset import LmdbDataset

import torch
from torch_geometric.data import Batch
from torch_geometric.loader.data_list_loader import DataListLoader

sys.path.append("/people/d3x771/projects/chemreasoner/chemreasoner/src")

from nnp.oc import OCAdsorptionCalculator

ads_calc = OCAdsorptionCalculator(
    **{
        "model": "gemnet-t",
        "traj_dir": Path("irrelevant"),
        "batch_size": 40,
        "device": "cpu",
        "ads_tag": 2,
        "fmax": 0.05,
        "steps": 300,
    }
)

#{
#        "model": "gemnet-oc-22",
#        "traj_dir": data_path,
#        "batch_size": 32,
#        "device": "cuda",
#        "ads_tag": 2,
#        "fmax": 0.03,
#        "steps": 200,
#    }

torch_calc = ads_calc.get_torch_model




/people/d3x771/.conda/envs/LLM/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Unknown option: -C
usage: git [--version] [--help] [-c name=value]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p|--paginate|--no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]


In [3]:
#module load cuda/11.8

In [4]:
#print(torch_calc.model)
print(torch_calc.model.num_blocks)
print(torch_calc.model.regress_forces)

3
True


In [5]:
# Question1: Map each sample GNN input to creating the descriptors per sample
# Question2: how do we get descriptor per sample
# Question 3: how do we implement initial ranking based on descriptors
# Question 4: Can Henry generate other configuration (cell shift, miller index) descriptors that are going into LLM

In [6]:
def get_per_sample_embeddings(output_embeddings, batch):
    """
    Given a dictionary comtaining model output per batch of the form:
    {"energy": E_t, "hidden_h":h, "hidden_m":m, 'edge_index':edge_index}
    
    generate, embeddings per model input:
    [embeddings_atomistic_graph1, embeddings_atomistic_graph2.....embeddings_atomistic_graphN]

    """
    data = output_embeddings
    #print(data)
    atom_emb = data['hidden_h']
    edge_emb = data['hidden_m']
    energies = data['energy']
    forces = data['forces']
    edge_index = data['edge_index']
    graph_embs = []
    for i in range(len(batch.ptr)-1):
        idx_start = batch.ptr[i]
        idx_end = batch.ptr[i+1]
        #print(i, idx_start, idx_end)
        graph_emb = atom_emb[idx_start:idx_end]
        #print(graph_emb.size())
        graph_emb = torch.mean(graph_emb, 0)
        #print(graph_emb.size())
        graph_embs.append(graph_emb)
    return(np.array(graph_embs))

In [7]:
import torch
import os
import os.path as osp
import pandas as pd
from tqdm import tqdm
from typing import Callable, List, Optional
from torch_geometric.data import Data, InMemoryDataset, download_url, extract_tar
import numpy as np
import lzma
import ase
from ase.io import iread
from ase.db import connect
from activate import data
import glob
import sys

In [8]:
"""
    data format
    z -> atomic numbers
    y -> prediction target/ total energy (optional here)
    pos -> coordinates of all atoms
    f -> forces per atom
    cell -> periodic boundary conditions for calculating neiborhood calculations
"""

'\n    data format\n    z -> atomic numbers\n    y -> prediction target/ total energy (optional here)\n    pos -> coordinates of all atoms\n    f -> forces per atom\n    cell -> periodic boundary conditions for calculating neiborhood calculations\n'

In [9]:
import oc20

dataset='OC20'


if(dataset == 'lmdb'):
    lmdb_dir='/qfs/projects/chemreasoner/test_lmdb/'
    dataset = LmdbDataset({"src": lmdb_dir})
    loader = DataListLoader(dataset, batch_size=20, shuffle=False)
elif(dataset == 'OC20'):
    datadir= '/qfs/projects/chemreasoner/data/OC20/'
    dataset = oc20.OC20(datadir, tag='200k')
    loader = DataListLoader(dataset, batch_size=256, shuffle=False)
else:
    print("UNSUPPORTED DATASET FORMAT, exiting...")
    sys.exit(1)

X = []
Y= []
for i, data_list in enumerate(loader):
    
    batch = Batch.from_data_list(data_list)
    
    print(i, len(batch))
    print(batch)
    #print(batch.ptr)
    if dataset =='lmdb':
        print(batch.descriptor)
    elif(dataset == 'OC20'):
        print(batch.name)
        batch.atomic_numbers = batch.z
    outputs = torch_calc.predict(batch,per_image=False)
    batch_embeddings = get_per_sample_embeddings(torch_calc.model.model_outemb, batch)
    batch_output = outputs['energy']
    #print(batch_embeddings.shap,batch_output.shape)
    for emb, out in zip(batch_embeddings, batch_output):
        X.append(emb)
        Y.append(out)

0 256
DataBatch(y=[256], pos=[18887, 3], z=[18887], f=[18887, 3], cell=[256, 3, 3], e_total=[256], e_ref=[256], rcell=[768, 3], name=[256], system_id=[256], idx=[256], natoms=[256], pbc=[768], batch=[18887], ptr=[257])


device 0:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]/people/d3x771/.conda/envs/LLM/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/qfs/people/d3x771/projects/chemreasoner/chemreasoner/ext/ocp/ocpmodels/models/gemnet/gemnet.py:373: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  neighbors_new // 2,
/qfs/people/d3x771/projects/chemreasoner/chemreasoner/ext/ocp/ocpmodels/models/gemnet/gemnet.py:471: UserW

1 256
DataBatch(y=[256], pos=[19108, 3], z=[19108], f=[19108, 3], cell=[256, 3, 3], e_total=[256], e_ref=[256], rcell=[768, 3], name=[256], system_id=[256], idx=[256], natoms=[256], pbc=[768], batch=[19108], ptr=[257])


device 0:   0%|                                                                                     | 0/1 [04:14<?, ?it/s]


2 256
DataBatch(y=[256], pos=[19478, 3], z=[19478], f=[19478, 3], cell=[256, 3, 3], e_total=[256], e_ref=[256], rcell=[768, 3], name=[256], system_id=[256], idx=[256], natoms=[256], pbc=[768], batch=[19478], ptr=[257])


device 0:   0%|                                                                                     | 0/1 [02:29<?, ?it/s]


KeyboardInterrupt: 

In [10]:

X = np.array(X)
Y= np.array(Y)
print(X.shape, Y.shape)
#X = np.reshape(X, (512, 512))
#Y = np.reshape(Y, (512, 1))
#print(X.shape, Y.shape)

(512, 512) (512, 1)


In [11]:
import numpy as np
import os
import pickle
import logging
import xgboost as xgb
 
class GBMRegressor:
    """
    Union approach for Gradient Boosting Machine uncertainty estimation
    from https://link.springer.com/article/10.1186/s13321-023-00753-5 
    """
    def __init__(self, savedir='./', lower_alpha=0.1, upper_alpha=0.9, n_estimators=100):
        """Initialize GBM regressors
        Args:
          savedir (str): Directory to save fit GBM regressors. 
                         (default: :obj:`./`)
          lower_alpha (float): The alpha-quantile of the quantile loss function.
                               Values must be in the range (0.0, 1.0). 
                               (default: :obj:`0.1`)
          upper_alpha (float): The alpha-quantile of the quantile loss function. 
                               Values must be in the range (0.0, 1.0). 
                               (default: :obj:`0.9`)
          n_estimators (int): The number of boosting stages to perform.
                              (default: :obj:`100`)
        """
        self.savedir = savedir
        self.alpha = np.array([lower_alpha, upper_alpha])
        self.n_estimators = n_estimators
        
    @property
    def model_file(self):
        return 'GBMRegressor.pkl'
 
        
    def update(self, embeddings, target):
        """Update GBM models after training epoch."""          
        Xy = xgb.QuantileDMatrix(embeddings, target)
        Xy_test = xgb.QuantileDMatrix(embeddings, target, ref=Xy)

        self.booster = xgb.train(
            {
                "objective": "reg:quantileerror",
                "tree_method": "hist",
                "quantile_alpha": self.alpha,
                "learning_rate": 0.04,
                "max_depth": 5,
                "verbosity": 0,
                "disable_default_eval_metric": True,
            },
            Xy,
            num_boost_round=self.n_estimators,
            )
 
    def predict(self, embeddings):
        """Predict uncertainties for set of embeddings."""
 
        scores = self.booster.inplace_predict(embeddings).T
        return np.abs(scores[0]-scores[1])/2
 
    def _save(self):
        """Save GBM regressor parameters to file."""
        with open(os.path.join(self.savedir, self.model_file), 'wb') as f:
            pickle.dump(self.booster, f)
 
 
    def _load(self):
        """Load trained GBM regressors from file."""
        if os.path.isfile(os.path.join(self.savedir, self.model_file)):
            with open(os.path.join(self.savedir, self.model_file), 'rb') as f:
                self.booster = pickle.load(f)
        else:
            logging.warning(f'No trained GBM regressor found in {self.savedir}. Call GBMRegressor.update to train a model.')

In [12]:
# install xgboost
uq_model = GBMRegressor()


In [13]:
import sklearn as sk

X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y , test_size=0.3, random_state=42)

uq_model.update(X_train,Y_train)

In [14]:
uq_model.predict(X_test)

array([1.6033977 , 0.73653156, 1.6619005 , 1.220981  , 0.9422441 ,
       1.1381476 , 0.65301645, 0.5406874 , 0.78450614, 1.2528241 ,
       1.248014  , 0.6767561 , 1.2372406 , 0.77329195, 1.3526403 ,
       0.9138219 , 0.84102476, 1.4724238 , 1.1552675 , 1.1156557 ,
       2.0222178 , 1.0507889 , 1.2224222 , 6.7936234 , 1.9063494 ,
       0.9781883 , 1.9560361 , 3.1249433 , 0.9493817 , 1.0077488 ,
       0.9555403 , 0.85769784, 1.2346307 , 0.6665735 , 1.753107  ,
       0.7544469 , 1.4823492 , 1.0629822 , 1.1034492 , 0.80066186,
       1.757617  , 2.3911357 , 0.35435978, 1.2724328 , 1.222982  ,
       1.237083  , 0.7011486 , 0.97969615, 0.29001102, 0.93491465,
       0.5383747 , 1.631411  , 0.84849364, 1.4128246 , 0.60093904,
       0.8664035 , 0.42883253, 0.75119936, 1.0361155 , 1.1000831 ,
       0.51049984, 0.81972146, 2.64171   , 0.81146026, 2.646253  ,
       0.6346876 , 0.6521261 , 0.7723559 , 1.5382522 , 1.277319  ,
       1.9854906 , 1.0271649 , 1.1764631 , 1.5450937 , 1.02421

In [ ]:
# plot X descriptors against the adsorption energy